# 填充图

本节介绍如何使用 cedarkit-maps 绘制 850 hPa 风速填充图。

首先导入需要使用的包。包括：

- 数据结构：numpy, pandas
- cedarkit 工具套件
    - 数据准备：reki
        - 数据查找函数： `find_local_file`
        - GRIB2文件要素加载函数：`load_field_from_file`
    - 气象可视化：cedarkit.maps
        - 绘图板：`Panel`
        - 底图布局：`EastAsiaMapDomain`
        - 绘图样式：`ContourStyle`
        - 颜色表加载函数：`get_ncl_colormap`

In [ ]:
import numpy as np
import pandas as pd

from reki.data_finder import find_local_file
from reki.format.grib.eccodes import load_field_from_file

from cedarkit.maps.style import ContourStyle
from cedarkit.maps.chart import Panel
from cedarkit.maps.domains import EastAsiaMapDomain
from cedarkit.maps.colormap import get_ncl_colormap

设置绘图的数据参数，使用 CMA-GFS 2024 年 4 月 1 日 00 时次 024 数据。

In [ ]:
system_name = "CMA-GFS"
data_type = "cma_gfs_gmf/grib2/orig"
start_time = pd.to_datetime("2024-04-01 00:00:00")
forecast_time = pd.to_timedelta("24h")

## 加载数据

获取 GRIB2 数据路径

In [ ]:
file_path = find_local_file(
    data_type,
    start_time=start_time,
    forecast_time=forecast_time,
)
file_path

加载 850 hPa 风场。风场在 ecCodes 的内置要素名为 `u` 和 `v`

In [ ]:
u_850_field = load_field_from_file(
    file_path,
    parameter="u",
    level_type="pl",
    level=850
)
v_850_field = load_field_from_file(
    file_path,
    parameter="v",
    level_type="pl",
    level=850
)
u_850_field

计算得到风速场

In [ ]:
wind_speed_850_field = np.sqrt(u_850_field ** 2 + v_850_field ** 2)
wind_speed_850_field

## 配置

定义填充图层次

In [ ]:
wind_speed_contour_lev = np.arange(8, 26, 2)

定义填充图色表

In [ ]:
color_index = np.arange(90, 236, 15) - 2
color_index[0] = -1
wind_speed_color_map = get_ncl_colormap(
    "rainbow+white+gray", index=color_index
)

定义填充图样式 (`ContourStyle`)

In [ ]:
wind_speed_style = ContourStyle(
    colors=wind_speed_color_map,
    levels=wind_speed_contour_lev,
    fill=True,
)

## 绘制

创建中国区域底图布局

In [ ]:
domain = EastAsiaMapDomain()

绘制填充图，设置标题和颜色条

In [ ]:
panel = Panel(domain=domain)
panel.plot(wind_speed_850_field, style=wind_speed_style)

domain.set_title(
    panel=panel,
    graph_name="850 hPa WIND (m/s) shadow",
    system_name=system_name,
    start_time=start_time,
    forecast_time=forecast_time,
)
domain.add_colorbar(panel=panel, style=wind_speed_style)
panel.show()